In [1]:
!pip list

Package            Version     Editable project location
------------------ ----------- ----------------------------------------------------------
aiohappyeyeballs   2.4.4
aiohttp            3.11.11
aiosignal          1.3.2
appnope            0.1.4
asttokens          3.0.0
asyncio            3.4.3
attrs              24.3.0
certifi            2024.12.14
charset-normalizer 3.4.1
cine_notes         0.0.0       /Users/prakharagarwal/Documents/Python Codes/CineNotes/src
comm               0.2.2
debugpy            1.8.11
decorator          5.1.1
exceptiongroup     1.2.2
executing          2.1.0
from-root          1.3.0
frozenlist         1.5.0
idna               3.10
imageio            2.36.1
imageio-ffmpeg     0.5.1
importlib_metadata 8.5.0
ipykernel          6.29.5
ipython            8.31.0
jedi               0.19.2
jupyter_client     8.6.3
jupyter_core       5.7.2
matplotlib-inline  0.1.7
moviepy            2.1.1
multidict          6.1.0
nest_asyncio       1.6.0
numpy              2.2.1
p

In [2]:
%pip install pydub 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pydub

In [6]:
from cine_notes.components.audio_transcript import HuggingFaceWhisperAPI

In [8]:
from pydub import AudioSegment
from pydub.utils import make_chunks

audio_file = r"/Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/audio.mp3"
audio_path = r"/Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/parts"
# Load the audio file
audio = AudioSegment.from_file(audio_file)

# Define chunk length in milliseconds (e.g., 10 seconds)
chunk_length_ms = 300000

# Split the audio into chunks
chunks = make_chunks(audio, chunk_length_ms)
chunk_list = []
# Export chunks as separate files
for i, chunk in enumerate(chunks):
    chunk_name = f"{audio_path}/chunk{i}.mp3"
    chunk_list.append(chunk_name)
    chunk.export(chunk_name, format="mp3")
    print(f"Exported {chunk_name}")

Exported /Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/parts/chunk0.mp3
Exported /Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/parts/chunk1.mp3
Exported /Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/parts/chunk2.mp3


In [9]:
type(chunks[0].raw_data)

bytes

In [10]:
type(chunk)

pydub.audio_segment.AudioSegment

In [12]:
whisper = HuggingFaceWhisperAPI()
audio_file = r"/Users/prakharagarwal/Documents/Python Codes/CineNotes/my_videos/Deep Learning for Computer Vision  CampusX Deep Learning Course  DLCV/audio/parts/chunk0.mp3"
transcribe = whisper.query(audio_file)

CineNotesException: Error occurred in python script: [/Users/prakharagarwal/Documents/Python Codes/CineNotes/src/cine_notes/components/audio_transcript.py] at line number: [117] with error: [Failed to query Hugging Face API
Detailed Exception:
This event loop is already running]

In [ ]:
%pip install asyncio aiohttp

In [ ]:
chunks_data = []
for chunk in chunk_list:
    with open(chunk, 'rb') as f:
        chunks_data.append(f.read())
        f.close()

In [ ]:
len(chunks_data)

In [ ]:
import asyncio
import aiohttp
from dotenv import load_dotenv
load_dotenv()
import os
hf_token = os.getenv("HUGGINGFACE_WHISPER_TOKEN")

class APIClient:
    def __init__(self, api_url, headers):
        self.api_url = api_url
        self.headers = headers

    async def send_request(self, session, data):
        """Send a POST request with the given payload."""
        try:
            async with session.post(self.api_url, headers=self.headers, data=data) as response:
                response.raise_for_status()
                result = await response.json()
                return {"status": response.status, "data": result}
        except aiohttp.ClientError as e:
            return {"error": str(e)}

    async def execute_requests(self, payloads):
        """Execute multiple POST requests in parallel."""
        async with aiohttp.ClientSession() as session:
            tasks = [self.send_request(session, payload) for payload in payloads]
            return await asyncio.gather(*tasks)

# Example usage
api_url = "https://api-inference.huggingface.co/models/openai/whisper-large-v3-turbo"
headers = {"Authorization": f"Bearer {hf_token}"}

# Different payloads for parallel execution
payloads = chunks_data

client = APIClient(api_url, headers)

async def main():
    results = await client.execute_requests(payloads)
    for idx, result in enumerate(results):
        print(f"Response {idx + 1}:\n {result}")
        print("-" * 50)

try:
    loop = asyncio.get_running_loop()
    # If already in a running event loop
    task = asyncio.create_task(main())
    loop.run_until_complete(task)
except RuntimeError:
    # If no event loop is running
    asyncio.run(main())


In [ ]:
import json
path = r'/Users/prakharagarwal/Documents/Python Codes/CineNotes/artifacts/2025_01_10__00_49_58/output.json'
# Load the JSON data
with open(path, 'r', encoding='utf-8') as infile:
    data = json.load(infile)

print(data)

# Save the JSON data with UTF-8 encoding
with open(path, 'w', encoding='utf-8') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=4)

print("File saved as 'output_readable.json' with readable characters.")